In [ ]:
# Import the necessary libraries
import math
from matplotlib import pyplot as plt
import torch
import gpytorch
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import pandas_ta as ta
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from plotly.subplots import make_subplots

from algrow import Algrow, TrainingData

In [ ]:
eth = TrainingData(symbol="ETHAUD", bar="1h", start_time="January 01, 2021")
eth.get_data()

In [ ]:
# A GP model with constant mean function and spectral mixture (SM) kernel
class SpectralMixtureGP(gpytorch.models.ExactGP):
    def __init__(self, x_train, y_train, likelihood):
        super(SpectralMixtureGP, self).__init__(x_train, y_train, likelihood)
        self.mean = gpytorch.means.ConstantMean() # Construct the mean function
        self.cov = gpytorch.kernels.SpectralMixtureKernel(num_mixtures=6) # Construct the kernel function
        self.cov.initialize_from_data(x_train, y_train) # Initialize the hyperparameters from data
        
    def forward(self, x):
        # Evaluate the mean and kernel function at x
        mean_x = self.mean(x)
        cov_x = self.cov(x)
        # Return the multivariate normal distribution using the evaluated mean and kernel function
        return gpytorch.distributions.MultivariateNormal(mean_x, cov_x) 
    

In [ ]:
# Indicators

# Predictive feature
pred_sma = 6

indicators = ta.Strategy(
    name="Momo and Volatility",
    description="Feature",
    ta=[
        {"kind": "sma", "length": pred_sma}
        ]
    )

eth.add_indicators(indicators)

In [ ]:
eth_features = ((eth.data[['SMA_6']]-eth.data[['SMA_6']].shift(+3))/eth.data[['SMA_6']].shift(+3))*100
eth.data.dropna(inplace=True)

In [ ]:
eth_sc = MinMaxScaler(feature_range=(0,1))
eth_scaled_features = eth_sc.fit_transform(eth_features)[:,0]
print(np.shape(eth_scaled_features))

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing = 0.01, row_heights=[0.7, 0.3])

bars = 750

# CANDLE STICK CHART
fig.add_trace(go.Candlestick(x=eth.data['closetime'][-bars:],
                open=eth.data['open'][-bars:],
                high=eth.data['high'][-bars:],
                low=eth.data['low'][-bars:],
                close=eth.data['close'][-bars:]), row=1, col=1)

# MOVING AVERAGES
fig.add_trace(go.Scatter(x=eth.data['closetime'][-bars:],
                         y=eth.data['SMA_6'][-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='SMA_3'), row=1, col=1)


# MOVING AVERAGES
fig.add_trace(go.Scatter(x=eth.data['closetime'][-bars:],
                         y=eth_scaled_features[-bars:],
                         opacity=0.7,
                         line=dict(width=2),
                         name='pred_feature'), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(height=700, width=850, title_text="Input Features")
fig.show()


In [ ]:
# The training data is 15 equally-spaced points from [0,1] 
x_train_full = torch.linspace(0, eth_scaled_features.size, eth_scaled_features.size)
x_train = torch.linspace(0, eth_scaled_features.size, eth_scaled_features.size)[-2000:-200]

# The true function is sin(2*pi*x) with Gaussian noise N(0, 0.04)
y_train_full = torch.tensor(eth_scaled_features).to(torch.float32)
y_train = torch.tensor(eth_scaled_features).to(torch.float32)[-2000:-200]


In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

# MOVING AVERAGES
fig.add_trace(go.Scatter(x=x_train,
                         y=y_train,
                         opacity=0.7,
                         line=dict(width=2),
                         name='SMA_3'), row=1, col=1)


fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(height=700, width=850, title_text="Input Features")
fig.show()


In [ ]:
# Initialize the likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = SpectralMixtureGP(x_train, y_train, likelihood)

In [ ]:
# Put the model into training mode
model.train()
likelihood.train()

# Use the Adam optimizer, with learning rate set to 0.1
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Use the negative marginal log-likelihood as the loss function
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# Set the number of training iterations
n_iter = 500

for i in range(n_iter):
    # Set the gradients from previous iteration to zero
    optimizer.zero_grad()
    # Output from model
    output = model(x_train)
    # Compute loss and backprop gradients
    loss = -mll(output, y_train)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, n_iter, loss.item()))
    optimizer.step()

In [ ]:
# The test data is 50 equally-spaced points from [0,5]
x_test = torch.linspace(17000, 18000, 1000)

# Put the model into evaluation mode
model.eval()
likelihood.eval()

# The gpytorch.settings.fast_pred_var flag activates LOVE (for fast variances)
# See https://arxiv.org/abs/1803.06058
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # Obtain the predictive mean and covariance matrix
    f_preds = model(x_test)
    f_mean = f_preds.mean
    f_cov = f_preds.covariance_matrix
    
    # Make predictions by feeding model through likelihood
    observed_pred = likelihood(model(x_test))
    lower, upper = observed_pred.confidence_region()
    



fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing = 0.01)

fig.add_trace(go.Scatter(x = x_train_full.numpy(),
                         y = y_train_full.numpy(),
                         line_color = 'blue',
                         line = {'dash': 'dash'},
                         name = 'truth',
                         opacity = 0.7),
              row = 1, col = 1)

fig.add_trace(go.Scatter(x = x_train.numpy(),
                         y = y_train.numpy(),
                         line_color = 'black',
                         line=dict(width=2),
                         name = 'truth',
                         opacity = 0.7),
              row = 1, col = 1)


# Lower Bound fill in between with parameter 'fill': 'tonexty'

fig.add_trace(go.Scatter(x = x_test.numpy(),
                         y = upper.numpy(),
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         name = 'upper',
                         opacity = 0.5),
              row = 1, col = 1)

fig.add_trace(go.Scatter(x = x_test.numpy(),
                         y = lower.numpy(),
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         fillcolor='rgba(150,26,65,0.1)',
                         name = 'lower',
                         opacity = 0.2),
              row = 1, col = 1)


fig.add_trace(go.Scatter(x = x_test.numpy(),
                         y = observed_pred.mean.numpy(),
                         line_color = 'red',
                         line=dict(width=2),
                         name = 'prediction',
                         opacity = 0.5),
              row = 1, col = 1)


fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(height=700, width=850, title_text="Input Features")
fig.show()
